In [3]:
!pip install ipython-autotime
!pip install transformers
!pip install sentencepiece

     |████████████████████████████████| 4.2 MB 9.6 MB/s 
     |████████████████████████████████| 84 kB 4.1 MB/s 
     |████████████████████████████████| 596 kB 87.3 MB/s 
     |████████████████████████████████| 6.6 MB 75.6 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 1.2 MB 8.7 MB/s 


In [4]:
from transformers import pipeline
import pandas as pd
import numpy as np
from scipy import stats
import operator
%load_ext autotime

time: 84.8 µs (started: 2022-05-20 09:53:08 +00:00)


In [5]:
classifier = pipeline("zero-shot-classification",
                      model="facebook/bart-large-mnli")

Downloading:   0%|          | 0.00/1.13k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.52G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

time: 34.9 s (started: 2022-05-20 09:53:13 +00:00)


In [6]:
df=pd.read_csv('/content/after_clustering_5_17.csv')

time: 16.1 ms (started: 2022-05-20 09:53:48 +00:00)


In [7]:
data=df[['preprocessed_body']].loc[df['Body_labels_8']==6]

time: 12.2 ms (started: 2022-05-20 09:53:48 +00:00)


In [5]:
data

,preprocessed_body
0,we be look for contract option for two hundred...
2,dear sir madam greeting from tata teleservices...
8,hi team this be jason smith associate with tdi...
19,hello please find my resume for the post of sr...
20,important note a per the government s new norm...
...,...
155,webinar cannot read or see image view this ema...
159,dear recruiter greeting for the day i hope thi...
167,hi there i hope the first quarter of the year ...
168,how to achieve cyber security job


time: 8.75 ms (started: 2022-05-20 07:21:41 +00:00)


In [18]:
def classify(text):
    res=classifier(text, candidate_labels)
    res_dict=dict()
    label=res['labels']
    score=res['scores']
    [res_dict.update({label[i]:score[i]}) for i in range(len(res['labels']))]
        

    return res_dict

candidate_labels = ['job inquiry', 'request for service', 'service provider']
data['intents']=data['preprocessed_body'].apply(lambda x: classify(x))


time: 4min 26s (started: 2022-05-20 07:38:22 +00:00)


In [19]:
data['fb_bart_intent']=data['intents'].apply(lambda x: max(x.items(), key=operator.itemgetter(1))[0])

time: 3.96 ms (started: 2022-05-20 07:55:46 +00:00)


Using the Roberta_large_Xnli_anli model 

In [9]:
classifier_roberta = pipeline("zero-shot-classification", 
                       model="vicgalle/xlm-roberta-large-xnli-anli")

Downloading:   0%|          | 0.00/734 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.09G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.83M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/150 [00:00<?, ?B/s]

time: 1min 1s (started: 2022-05-20 09:54:31 +00:00)


In [8]:
def classify_r(text):
    res=classifier_roberta(text, candidate_labels)
    res_dict=dict()
    label=res['labels']
    score=res['scores']
    [res_dict.update({label[i]:score[i]}) for i in range(len(res['labels']))]
        

    return res_dict

candidate_labels = ['job inquiry', 'request for service', 'service provider']
data['intents_r']=data['preprocessed_body'].apply(lambda x: classify_r(x))

time: 3min 27s (started: 2022-05-20 07:24:00 +00:00)


In [10]:
data['Roberta_large_xnli_intent']=data['intents_r'].apply(lambda x: max(x.items(), key=operator.itemgetter(1))[0])

time: 3.17 ms (started: 2022-05-20 07:28:31 +00:00)


In [11]:
data

,preprocessed_body,intents_r,Roberta_large_xnli_intent
0,we be look for contract option for two hundred...,"{'request for service': 0.9534129500389099, 's...",request for service
2,dear sir madam greeting from tata teleservices...,"{'service provider': 0.5985546708106995, 'requ...",service provider
8,hi team this be jason smith associate with tdi...,"{'job inquiry': 0.627028226852417, 'request fo...",job inquiry
19,hello please find my resume for the post of sr...,"{'job inquiry': 0.8636243343353271, 'service p...",job inquiry
20,important note a per the government s new norm...,"{'service provider': 0.9290523529052734, 'requ...",service provider
...,...,...,...
155,webinar cannot read or see image view this ema...,"{'request for service': 0.7344494462013245, 's...",request for service
159,dear recruiter greeting for the day i hope thi...,"{'job inquiry': 0.5293307304382324, 'service p...",job inquiry
167,hi there i hope the first quarter of the year ...,"{'request for service': 0.4318058490753174, 'j...",request for service
168,how to achieve cyber security job,"{'job inquiry': 0.5866370797157288, 'request f...",job inquiry


time: 15.3 ms (started: 2022-05-20 07:28:38 +00:00)


 Using the distilbart-mnli-12-3 model

In [8]:
classifier_distilbart=pipeline("zero-shot-classification", 
                       model="valhalla/distilbart-mnli-12-3")

Downloading:   0%|          | 0.00/1.36k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/977M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/772 [00:00<?, ?B/s]

time: 24.1 s (started: 2022-05-20 09:54:07 +00:00)


In [13]:
def classify_d(text):
    res=classifier_distilbart(text, candidate_labels)
    res_dict=dict()
    label=res['labels']
    score=res['scores']
    [res_dict.update({label[i]:score[i]}) for i in range(len(res['labels']))]
        

    return res_dict

candidate_labels = ['job inquiry', 'request for service', 'service provider']
data['intents_d']=data['preprocessed_body'].apply(lambda x: classify_d(x))

time: 2min 27s (started: 2022-05-20 07:34:35 +00:00)


In [14]:
data['distilbart_intent']=data['intents_d'].apply(lambda x: max(x.items(), key=operator.itemgetter(1))[0])

time: 2.78 ms (started: 2022-05-20 07:37:35 +00:00)


In [21]:
data.drop(['intents_r','intents','intents_d'], axis=1, inplace=True)

time: 2.54 ms (started: 2022-05-20 07:56:03 +00:00)


In [23]:
data.to_csv("Intents_zeroshot.csv")

time: 5.46 ms (started: 2022-05-20 08:00:29 +00:00)


In [24]:
data


,preprocessed_body,Roberta_large_xnli_intent,distilbart_intent,fb_bart_intent
0,we be look for contract option for two hundred...,request for service,request for service,request for service
2,dear sir madam greeting from tata teleservices...,service provider,service provider,service provider
8,hi team this be jason smith associate with tdi...,job inquiry,job inquiry,job inquiry
19,hello please find my resume for the post of sr...,job inquiry,job inquiry,job inquiry
20,important note a per the government s new norm...,service provider,request for service,request for service
...,...,...,...,...
155,webinar cannot read or see image view this ema...,request for service,service provider,request for service
159,dear recruiter greeting for the day i hope thi...,job inquiry,job inquiry,job inquiry
167,hi there i hope the first quarter of the year ...,request for service,request for service,request for service
168,how to achieve cyber security job,job inquiry,job inquiry,job inquiry


time: 14.2 ms (started: 2022-05-20 08:05:40 +00:00)


In [11]:
candidate_labels = ['job inquiry', 'request for service', 'service provider']
text="hi greeting from isquare business solution i hope you be do well in this tumultuous time a node j and react j developer with four year of experience be available right now i have attach the profile and portfolio for your review this will be on a contract basis company to company remote let me know if that suit you please find the rate card below for monthly charge of twenty-two day one hundred and seventy-six hour one name ramakrishna p two experience four year three join immediate four skill html cs react j redux and javascript aws five charge one seventy zero gst month one name praveen kumar s two experience four year three join immediate four skill html cs react j node j express j mongo db dynamo db redux aws five charge one seventy zero gst month thanks and regard mohammed faizan s business development executive m ninety-one eight hundred and one two hundred five thousand madurai tn india"
classifier_distilbart(text, candidate_labels)

{'labels': ['job inquiry', 'request for service', 'service provider'],
 'scores': [0.567954957485199, 0.3300318121910095, 0.10201319307088852],
 'sequence': 'hi greeting from isquare business solution i hope you be do well in this tumultuous time a node j and react j developer with four year of experience be available right now i have attach the profile and portfolio for your review this will be on a contract basis company to company remote let me know if that suit you please find the rate card below for monthly charge of twenty-two day one hundred and seventy-six hour one name ramakrishna p two experience four year three join immediate four skill html cs react j redux and javascript aws five charge one seventy zero gst month one name praveen kumar s two experience four year three join immediate four skill html cs react j node j express j mongo db dynamo db redux aws five charge one seventy zero gst month thanks and regard mohammed faizan s business development executive m ninety-one e

time: 2.56 s (started: 2022-05-20 09:55:56 +00:00)


In [12]:
text="hi greeting from isquare business solution i hope you be do well in this tumultuous time a node j and react j developer with four year of experience be available right now i have attach the profile and portfolio for your review this will be on a contract basis company to company remote let me know if that suit you please find the rate card below for monthly charge of twenty-two day one hundred and seventy-six hour one name ramakrishna p two experience four year three join immediate four skill html cs react j redux and javascript aws five charge one seventy zero gst month one name praveen kumar s two experience four year three join immediate four skill html cs react j node j express j mongo db dynamo db redux aws five charge one seventy zero gst month thanks and regard mohammed faizan s business development executive m ninety-one eight hundred and one two hundred five thousand madurai tn india"
classifier_roberta(text, candidate_labels)

{'labels': ['service provider', 'request for service', 'job inquiry'],
 'scores': [0.48858797550201416, 0.42307522892951965, 0.08833684772253036],
 'sequence': 'hi greeting from isquare business solution i hope you be do well in this tumultuous time a node j and react j developer with four year of experience be available right now i have attach the profile and portfolio for your review this will be on a contract basis company to company remote let me know if that suit you please find the rate card below for monthly charge of twenty-two day one hundred and seventy-six hour one name ramakrishna p two experience four year three join immediate four skill html cs react j redux and javascript aws five charge one seventy zero gst month one name praveen kumar s two experience four year three join immediate four skill html cs react j node j express j mongo db dynamo db redux aws five charge one seventy zero gst month thanks and regard mohammed faizan s business development executive m ninety-on

time: 3.8 s (started: 2022-05-20 09:56:00 +00:00)


In [13]:
classifier(text, candidate_labels)

{'labels': ['job inquiry', 'request for service', 'service provider'],
 'scores': [0.5318430066108704, 0.32959452271461487, 0.13856250047683716],
 'sequence': 'hi greeting from isquare business solution i hope you be do well in this tumultuous time a node j and react j developer with four year of experience be available right now i have attach the profile and portfolio for your review this will be on a contract basis company to company remote let me know if that suit you please find the rate card below for monthly charge of twenty-two day one hundred and seventy-six hour one name ramakrishna p two experience four year three join immediate four skill html cs react j redux and javascript aws five charge one seventy zero gst month one name praveen kumar s two experience four year three join immediate four skill html cs react j node j express j mongo db dynamo db redux aws five charge one seventy zero gst month thanks and regard mohammed faizan s business development executive m ninety-one

time: 4.35 s (started: 2022-05-20 09:56:22 +00:00)
